# Take the Tour

> Note: This tour was gerneated from a jupyter notebook. <br>
> [It is available here](https://github.com/mar10/nutree/blob/main/docs/jupyter/ug_tour.ipynb) 
> and can be used for interactive experiments.

Nutree organizes arbitrary object instances in an unobtrusive way. <br>
That means, we can add existing objects without having to derive from a common 
base class or implement a specific protocol.

In [51]:
from nutree import Tree

tree = Tree("Hello")
tree.add("N").add("u").up(2).add("T").add("r").up().add("ee")
tree.print()

Tree<'Hello'>
├── 'N'
│   ╰── 'u'
╰── 'T'
    ├── 'r'
    ╰── 'ee'


Strings can be added to a tree, but in a real-world scenario we want need to 
handle ordinary objects:

## Setup some sample classes and objects
Let's define a simple class hierarchy

In [52]:
import uuid


class Department:
    def __init__(self, name: str):
        self.guid = uuid.uuid4()
        self.name = name

    def __str__(self):
        return f"Department<{self.name}>"


class Person:
    def __init__(self, name: str, age: int):
        self.guid = uuid.uuid4()
        self.name = name
        self.age = age

    def __str__(self):
        return f"Person<{self.name} ({self.age})>"

and create some instances

In [53]:
development_dep = Department("Development")
test__dep = Department("Test")
marketing_dep = Department("Marketing")

alice = Person("Alice", 25)
bob = Person("Bob", 35)
claire = Person("Claire", 45)
dave = Person("Dave", 55)

Now that we have a bunch of instances, let's organize these objects in a 
hierarchical structure using [nutree](https://nutree.readthedocs.io/):

In [54]:
from nutree import Tree

tree = Tree("Organization")

dev_node = tree.add(development_dep)
test_node = dev_node.add(test__dep)
mkt_node = tree.add(marketing_dep)

tree.add(alice)
dev_node.add(bob)
test_node.add(claire)
mkt_node.add(dave)

tree.print()

Tree<'Organization'>
├── <__main__.Department object at 0x10e5a1670>
│   ├── <__main__.Department object at 0x10e59d670>
│   │   ╰── <__main__.Person object at 0x10e5a1850>
│   ╰── <__main__.Person object at 0x10e5a07a0>
├── <__main__.Department object at 0x10e5a0230>
│   ╰── <__main__.Person object at 0x10e5a0560>
╰── <__main__.Person object at 0x10e5a0200>


Tree nodes store a reference to the object in the `node.data` attribute.

The nodes are formatted for display by the object's  `__repr__` implementation 
by default. <br>
We can overide this by passing an 
[f-string](https://docs.python.org/3/tutorial/inputoutput.html#formatted-string-literals) 
as `repr` argument:

In [55]:
tree.print(repr="{node.data}")

Tree<'Organization'>
├── Department<Development>
│   ├── Department<Test>
│   │   ╰── Person<Claire (45)>
│   ╰── Person<Bob (35)>
├── Department<Marketing>
│   ╰── Person<Dave (55)>
╰── Person<Alice (25)>


## Access Nodes
We can use the index syntax to get the node object for a given data object:

In [56]:
tree[alice]

Node<'Person<Alice (25)>', data_id=283484192>

In [57]:
assert tree[alice].data is alice, "nodes store objects in data attribute"

## Iteration and Searching

There are multiple methods to iterate the tree.

In [58]:
res = []
for node in tree:  # depth-first, pre-orde traversal
    res.append(node.data.name)
print(res)

['Development', 'Test', 'Claire', 'Bob', 'Marketing', 'Dave', 'Alice']


In [59]:
from nutree import IterMethod

res = []
for node in tree.iterator(method=IterMethod.POST_ORDER):
    res.append(node.data.name)
print(res)

['Claire', 'Test', 'Bob', 'Development', 'Dave', 'Marketing', 'Alice']


In [60]:
tree.visit(lambda node, memo: print(node.data.name), method=IterMethod.LEVEL_ORDER)

Development
Marketing
Alice
Test
Bob
Dave
Claire


## Mutation

## Data IDs and Clones

In [61]:
tree.print(repr="{node}", title=False)

Node<'Department<Development>', data_id=283484519>
├── Node<'Department<Test>', data_id=283483495>
│   ╰── Node<'Person<Claire (45)>', data_id=283484549>
╰── Node<'Person<Bob (35)>', data_id=283484282>
Node<'Department<Marketing>', data_id=283484195>
╰── Node<'Person<Dave (55)>', data_id=283484246>
Node<'Person<Alice (25)>', data_id=283484192>


## Serialization

In [62]:
tree.to_dict_list()

[{'data': 'Department<Development>',
  'children': [{'data': 'Department<Test>',
    'children': [{'data': 'Person<Claire (45)>'}]},
   {'data': 'Person<Bob (35)>'}]},
 {'data': 'Department<Marketing>',
  'children': [{'data': 'Person<Dave (55)>'}]},
 {'data': 'Person<Alice (25)>'}]

In [63]:
list(tree.to_list_iter())

[(0, {}), (1, {}), (2, {}), (1, {}), (0, {}), (5, {}), (0, {})]

In [64]:
t = Tree._from_list([(0, "A"), (0, "B"), (1, "C"), (0, "D"), (3, "E")])
print(t.format())

Tree<'4535745808'>
├── 'A'
│   ╰── 'C'
│       ╰── 'E'
├── 'B'
╰── 'D'


## Special Data Types
### Plain Strings

We can add simple string objects the same way as any other object

In [65]:
tree_str = Tree()
a = tree_str.add("A")
a.add("a1")
a.add("a2")
tree_str.add("B")
tree_str.print()

Tree<'4535754704'>
├── 'A'
│   ├── 'a1'
│   ╰── 'a2'
╰── 'B'


### Dictionaries

We cannot add Python `dict` objects to a tree, because nutree cannot derive
a *data_id* for unhashable types. <br>
A a workaround, we can wrap it inside `DictWrapper` objects:

In [66]:
from nutree import DictWrapper, Tree

d = {"title": "foo", "id": 1}

tree = Tree()
tree.add("A").up().add("B")
tree["A"].add(DictWrapper(d))
tree["B"].add(DictWrapper(d))
tree.print(repr="{node}")
# tree.find(d)

Tree<'4535739136'>
├── Node<'A', data_id=8885459032692883038>
│   ╰── Node<"DictWrapper<{'title': 'foo', 'id': 1}>", data_id=4534073472>
╰── Node<'B', data_id=7589688033162088899>
    ╰── Node<"DictWrapper<{'title': 'foo', 'id': 1}>", data_id=4534073472>


## Advanced

### Chaining

Some methods return a node instance, so we can chain calls. <br>
This allows for a more compact code and avoids some temporary variables:

In [67]:
Tree().add("A").add("a1").up().add("a2").up(2).add("B").tree.print()

Tree<'4535747360'>
├── 'A'
│   ├── 'a1'
│   ╰── 'a2'
╰── 'B'
